<a href="https://colab.research.google.com/github/MeldaBarrazaVirgilio/CCDProject/blob/main/Lesson%209%20-%20Python%20Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/christopherhuntley/data5405/blob/main/img/Dolan.png?raw=true" width="180px" align="right">

# **Lesson 9: Dictionaries**
_Associative arrays by another name_

## **Learning Objectives**

### Theory / Be able to explain ...
- The purpose and usage of associative arrays
- Python dictionaries as associative arrays
- Hashing and it's implications for dictionary keys

### Skills / Know how to  ...
- Display the hash for any dictionary key
- Iterate over dictionary items, keys, and values
- Generate dictionaries from keys and values
- Use a dictionary comprehension for efficient dictionary generation
---

## **The Magic of Associative Arrays**
>"A very little key will open a very heavy door." -― Charles Dickens, _Hunted Down_

After so many years of programming in C, I found myself using it for basically everything, until one day in 1994 I was asked by a very wise boss to try [AWK](https://en.wikipedia.org/wiki/AWK). AWK was a text processing language developed at Bell Labs in the 1970s by the same team that created C and Unix. It was designed to be a tiny domain-specific language for working with streaming text data. One would feed data to an AWK script one line at a time. AWK would then output text to an output file, also one line at a time. It could, of course, remember things from one line to the next, allowing it to accumulate information along the way.

Soon I was using AWK for lots of text processing tasks. One notable application was to translate mainframe data into SQL code for loading into a relational database. Data would come in one line at a time and then go right into the database. I think I got at least one promotion from just this one parlor trick. A year or so later, in late 1995 or early 1996, I used the same trick to develop a dashboarding web app that was cobbled together with AWK and bash scripts. No Perl. No Python. No PHP. No Java. Just AWK and bash on a Unix command line. I am still amazed that it worked but we never had a crash or any other bug reported.   

One of the reasons why I loved AWK so much was a feature called "associative arrays" where we could index a variable length array with text **keys** instead of integers. We could even mix keys with integer indexes if we liked. This meant that, for example, I could have an array of birthdays indexed by people's names. Or vice versa if that was what I wanted.  Or, I could create a histogram for words in a file with two lines of code. The potential uses seemed endless. Nothing could have been more convenient for a wannabe smart and lazy programmer.

The Python equivalent of an associative array is a **dictionary**. It does many of the same things as a list but with keys instead of positions. Like associative arrays, there are an endless array of uses. If you have ever pulled data from a web API or added a Series to a DataFrame then you have used something like a dictionary. It's just how it's done.

---
## **Dictionaries as Collections of Key-Value Pairs**
Python dictionaries have the type `dict`. Here's a brief example, followed by a few notes.

In [ ]:
birthdays = {'Washington':'1732-02-22','Jefferson':'1743-04-13','Lincoln':'1809-02-12'}
birthdays['Madison']='1751-03-16'
for president in birthdays:
    print(president,"was born", birthdays[president])

Washington was born 1732-02-22
Jefferson was born 1743-04-13
Lincoln was born 1809-02-12
Madison was born 1751-03-16


- `dict` literals work like `list` literals except they use curly brackets `{}` instead of square brackets `[]`.
- `dict` indexes use **keys** of any **Hashable** type (more about this in a minute) instead of just integers.
- Each key-value pair is literally encoded as `key : value`. if a key is repeated within a dictionary then the value is replaced with each time with the newest value.
- The bracket operator `[]` is used for retrieving specific values, just like a list.
- Dictionaries are mutable. We can add or remove key-value pairs as needed. The `+=` operator doesn't work though. Use the `[ ]` operator instead.

In [ ]:
birthdays += {'Adams Sr.':'1735-10-30'}

TypeError: unsupported operand type(s) for +=: 'dict' and 'dict'

In [ ]:
birthdays['Adams Sr.'] = '1735-10-30'
birthdays

{'Washington': '1732-02-22',
 'Jefferson': '1743-04-13',
 'Lincoln': '1809-02-12',
 'Madison': '1751-03-16',
 'Adams Sr.': '1735-10-30'}

## **An Aside about Sets**

The built-in `set` data type is one of the hidden gems in the Python standard library.

A `set` is like a `list` but ... :
- With no duplicate items
- No implied ordering (and thus no `[ ]` indexing or slicing)

Here are a couple examples:

In [ ]:
# a list of numbers
{1,2,2,3}

{1, 2, 3}

In [ ]:
# that has no implied ordering
{1,2,2,3}[1]

<>:2: SyntaxWarning: 'set' object is not subscriptable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'set' object is not subscriptable; perhaps you missed a comma?
<ipython-input-9-92f9f86a2d70>:2: SyntaxWarning: 'set' object is not subscriptable; perhaps you missed a comma?
  {1,2,2,3}[1]


TypeError: 'set' object is not subscriptable

You may be telling yourself "Okay, so what? It seems like we could just use a list for that."

They key here is that sets enforce some rules that are very useful for certain kinds of data analysis.
- The no duplicates rule allows us to construct sets of **keys** without accidentally creating duplicates. So where might we care about that? With `dict` keys, for example.
- The lack of ordering forces us to go beyond indexed (`[ ]`) access to items and instead use a proper iterator. It's just more Pythonic and less prone to certain classes of bugs.

> **Heads Up:** The standard set iterator (used by `for` loops, etc.) traverses the items in ascending order. In other words `list(set( ... ))` is the same as `sorted(list(set( ... )))`. This can be useful if you want to treat a set like a list without having to sort it first.

### **Hashing**
To ensure data integrity, dictionary keys (and `set` items) are required to be:
- **Unique**: If two items have the same key, then how do we know which is which?
- **Immutable**: If we can change the value of a key (e.g., via aliasing) then how does the dictionary let everybody know about it?
- **Printable**: If not printable/visible, then how can we humans use them safely?

When passed an object, a **hashing** function generates a _printable_ **hash** or **digest** value that is _almost_ guaranteed to be unique. The odds of "collision" (i.e., two objects with the same hash) is very, very, very remote. Further, if the object being hashed is itself immutable then we have met all three requirements for dictionary keys:

1. Each key has a unique hash. If two keys are the same then they generate the same hash.
2. Because the key is required to be immutable, then so is the hash.
3. Hashes are printable as (typically) very long strings of characters or digits. So, even if the key itself isn't printable, its hash is.

Besides its obvious integrity advantages, hashing of keys is also highly efficient. Since hashes are convertible to strings or integers, we can sort them just like list positions. That makes using a key to lookup a value just as efficient as using an integer index to look up a value in a list. (Ever used a primary key or index to speed up a SQL query? That's exactly the same thing.)

While the precise hashing function may vary from data type to data type, the default implementation uses a version of the Fowler-Noll-Vo algorithm which is outside the scope of this course. However, we can call the `hash()` standard library function on any immutable object with 100% predictable results:   

In [ ]:
print(hash( 1 ))                         # int
print(hash( 2.3 ))                       # float
print(hash( "Mary Had a Little Lamb" ))  # string
print(hash( b'Mary Had a Little Lamb' )) # bytes (same as string)
print(hash( (1,2,3) ))                   # tuple, which is immutable
print(hash( hash ))                      # the hash function object
print(hash( [1,2,3] ))                   # list; oops that's mutable!

1
691752902764107778
4190155967270737076
4190155967270737076
529344067295497451
2468734438158


TypeError: unhashable type: 'list'

### **Dictionary Traversal**
When iterating over a `dict`, we can use one of three iterator **_view methods_** that return list-like sequences:
- `keys()` which returns all keys
- `values()` which returns all the values
- `items()` which returns all the key-value pairs (a.k.a., "items")

When used in a `for` loop the default is to use the `keys()` iterator:

In [ ]:
# the default iteration order
# birthdays is still a dict indexed by last name
for president in birthdays:
    print(president,"was born", birthdays[president])
print("---")
# explicitly iterating over keys()
for key in birthdays.keys():
    print(key,"was born", birthdays[president])

Washington was born 1732-02-22
Jefferson was born 1743-04-13
Lincoln was born 1809-02-12
Madison was born 1751-03-16
Adams Sr. was born 1735-10-30
---
Washington was born 1735-10-30
Jefferson was born 1735-10-30
Lincoln was born 1735-10-30
Madison was born 1735-10-30
Adams Sr. was born 1735-10-30


However, we can also iterate over items or even values, though with somewhat differing results.

In [ ]:
# iterating over items; each item is a tuple
for item in birthdays.items():
    print(item)
print("---")
# iterating over values()
for v in birthdays.values():
    print(v)

('Washington', '1732-02-22')
('Jefferson', '1743-04-13')
('Lincoln', '1809-02-12')
('Madison', '1751-03-16')
('Adams Sr.', '1735-10-30')
---
1732-02-22
1743-04-13
1809-02-12
1751-03-16
1735-10-30


You may have noticed that the order is the same each time. As of Python 3.6, each iterator will always follow the order in which the keys were inserted into the dictionary.

### **Pulse Check ...**
**Use the [`dict()` function](https://docs.python.org/3/library/stdtypes.html#dict) to create a new dictionary called `presidents` that swaps the keys and values of the `birthdays` dictionary.** Each key should be a birthdate and each value should be the associated president's last name.

In [ ]:
# YOUR CODE HERE

In [ ]:
#@title A two-line solution

# Swap each key and value with a list comprehension of pairs
items = [ [b[1], b[0]] for b in birthdays.items() ]

presidents = dict([ [b[0],b[1]] for b in items ])
presidents

{'1732-02-22': 'Washington',
 '1743-04-13': 'Jefferson',
 '1809-02-12': 'Lincoln',
 '1751-03-16': 'Madison',
 '1735-10-30': 'Adams Sr.'}

In [ ]:
#@title A more compact 1-line solution

# All on one line
presidents = dict([ [b[1],b[0]] for b in birthdays.items() ])
presidents

{'1732-02-22': 'Washington',
 '1743-04-13': 'Jefferson',
 '1809-02-12': 'Lincoln',
 '1751-03-16': 'Madison',
 '1735-10-30': 'Adams Sr.'}

### **Producing Dictionaries from Existing Data**
In the examples so far, all of our dictionaries have been created as literals with `{}` or through the `dict()` function. However, dictionaries can be created in lots of curious ways. Just about any iterative process that generates paired sequences of keys and values can be used to create and populate a dictionary.

In [ ]:
d_keys =   ["Washington","Jefferson"]
d_values = ['1732-02-22','1743-04-13']

d = {}        # an empty dictionary
for i in range(len(d_keys)):
    d[d_keys[i]] = d_values[i]
d

{'Washington': '1732-02-22', 'Jefferson': '1743-04-13'}

While straightforward, this is not the most efficient way to generate a dictionary. There are actually two different one line equivalents that are both less code and more efficient. Both are explained below.

### **`dict` Comprehensions**
A dictionary comprehension is a lot like a list comprehension, which we covered in Lesson 8. The syntax is like this:
```python
{ key : value for item in items }
```
The key and/or value will vary from item to item.

In [ ]:
# reuses the d_keys and d_values from before

{ d_keys[i] : d_values[i] for i in range(len(d_keys)) }

{'Washington': '1732-02-22', 'Jefferson': '1743-04-13'}

There are other allowed forms (e.g., the pairs can be specified as tuples) but this is the most commonly used one.

### **That One Weird Zip Dict Trick (Say that fast 3 times)**
The `zip()` function converts several sequences of the same length into an iterator of tuples (immutable lists, covered in Lesson 10), where the each tuple is composed of corresponding items.

In [ ]:
bdays = ['1732-02-22','1743-04-13','1809-02-12']
presidents = ['Washington','Jefferson','Lincoln']

z = zip(bdays,presidents)  # z is an iterator
list(z)

[('1732-02-22', 'Washington'),
 ('1743-04-13', 'Jefferson'),
 ('1809-02-12', 'Lincoln')]

This can be very useful for generating dictionaries. Let one of the sequences be a list of keys and the other a list of values. When used with the `dict()` constructor we now have a quick and efficient way to zip the keys and values together into a single dict.

In [ ]:
# bdays is the keys list
# presidents is the values list
dict(zip(bdays,presidents))  # Voila! a one line dict maker

{'1732-02-22': 'Washington',
 '1743-04-13': 'Jefferson',
 '1809-02-12': 'Lincoln'}

---
## **Before you go ... Save your notebook to be sure it is up to date.**

---
> ## Every Tee Shirt Has a Story
> ABOUT THE SLASHDOT EFFECT     
> I discovered Slashdot my first semester at Fairfield in 1997. I was trying to figure out how to explain geek culture to my undergrad business students. "You mean you actually call yourselves geeks? ..." Why yes we did, and there was even this new website called Slashdot that billed itself as "News for Nerds" that was catching on with programmers around the world. Even the name was total geek, a play on how web newbies would read URLs aloud. The full URL was `http://slashot.org`. Try saying that out loud (in English) including punctuation, a couple times. It's quite the tongue twister.  
>
> In keeping with its motto, it was a sort of curated blog with open discussion (without a log in required) that linked to the freshest nerd content from around the web. Within a few months of its founding, it had a continuous, 24x7 readership in the millions, all very geeky and loyal. This was at a time when getting a few thousand people to visit a major media site was a big deal.
>
> Slashdot became famous with the non-geek crowd when the blog postings began linking to not-so-nerdy newspapers, magazines, political candidates, and other media providers that were not prepared for **2 million** geeks clicking the link at _exactly_ the same time. Servers would fry and then seize up for good, leaving people wondering what the heck just happened and asking about backups. Such was the [Slashdot Effect](https://en.wikipedia.org/wiki/Slashdot_effect).  
>
> I picked up this tee at a Linux World show circa 2006. By then Slashdot had faded a bit, though was still popular in the right circles. The site is still around, though with a much, much smaller readership. It lost a lot of the original geek readership when magazine publisher Condé Nast bought it around the time I picked up the tee shirt.   
>
> ![L9 Tee Front](https://github.com/christopherhuntley/data5405/blob/main/Photos/L09_TeeFront.jpeg?raw=true)

## Copyright &copy; 2020 Christopher Huntley. All rights reserved.